# Test env

In [32]:

#imports
import os
import streamlit as st
from PIL import Image
from loguru import logger
# local imports
from ingest.ingester import Ingester
from query.querier import Querier
from summarize.summarizer import Summarizer
import settings
import utils as ut
from query.querier import EnumMode
from ingest.ingester import IngestionMode


In [33]:
SYSTEM_PROMPT= """
### OBJECTIVE ###
Je bent een assistent voor de rijksoverheid. Jouw taak is om vragen te beantwoorden in het Nederlands. Zorg ervoor dat je alleen antwoord geeft op basis van de beschikbare context en dat je daar ook naar verwijst in je antwoord.

### AUDIENCE ###
De doelgroep van jouw antwoorden zijn ambtenaren. Geef alle relevante informatie uit de context, antwoord in het Nederlands leg in maximaal 100 woorden zoveel mogelijk uit.

### GUARDRAILS ###
Indien de context onvoldoende informatie bevat om de vraag te beantwoorden, verzin dan geen informatie maar geef aan dat er onvoldoende informatie beschikbaar is.

### INSTRUCTIONS ###
- Beantwoord de vraag altijd in het Nederlands, zelfs als de context in het Engels is gesteld.
- Vermijd het herhalen van de vraag in het antwoord en het herhalen van de instructies. Voer de instructies uit en geef een concreet antwoord op de gestelde vraag.
- Geef een stapsgewijze redenering bij het beantwoorden van de vraag en refereer naar specifieke zinnen uit de context die hebben bijgedragen aan het antwoord.
- Houd je antwoord nauw verbonden met de context en vermijd het toevoegen van informatie die niet expliciet in de context wordt vermeld.

- Voor meer informatie over de context, zeg het bestandsnaam die gevonden is in de source_document. Mits deze beschikbaar is.
### QUESTION ### \n
"""

In [34]:
# Commented the settings that arent used as parameters in the functions

# DOC_DIR = "./docs"
# CHUNK_DIR = "./chunks"
# VECDB_DIR = "./vector_stores"
# EVAL_DIR = "./evaluate"
# EVAL_APP_HEADER = "Evaluation"
# EVAL_APP_INFO = "./info/evaluation_explanation.txt"
# EVAL_FILE_NAME = "eval.json"
# CHAIN_VERBOSITY = False
LLM_TYPE = "local_llm"
LLM_MODEL_TYPE = "gemma2"
# API_URL = "http://127.0.0.1:11434"
AZUREOPENAI_API_VERSION = "2023-08-01-preview"
EMBEDDINGS_PROVIDER = "local_embeddings"
EMBEDDINGS_MODEL = "textgain/allnli-GroNLP-bert-base-dutch-cased"
TEXT_SPLITTER_METHOD = "NLTKTextSplitter"
# CHAIN_NAME = "conversationalretrievalchain"
# CHAIN_TYPE = "stuff"
# SEARCH_TYPE = "similarity"
# SCORE_THRESHOLD = 0.5
VECDB_TYPE = "chromadb"
CHUNK_SIZE = 1024
# CHUNK_K = 4
CHUNK_OVERLAP = 256
# RETRIEVAL_METHOD = "regular"


folderSelected = "kamerVragen"
my_folder_path_selected, my_vectordb_folder_path_selected = ut.create_vectordb_name(folderSelected)


In [35]:
# Init
querier = Querier(
  llm_type=LLM_TYPE, 
  llm_model_type=LLM_MODEL_TYPE, 
  embeddings_model=EMBEDDINGS_MODEL, 
  embeddings_provider=EMBEDDINGS_PROVIDER, 
  azureopenai_api_version=AZUREOPENAI_API_VERSION
  )

ingester = Ingester(
  collection_name=folderSelected, 
  content_folder=my_folder_path_selected, 
  vectordb_folder=my_vectordb_folder_path_selected,
  embeddings_model=EMBEDDINGS_MODEL,
  text_splitter_method=TEXT_SPLITTER_METHOD,
  chunk_size=CHUNK_SIZE,
  chunk_overlap=CHUNK_OVERLAP,
  vecdb_type=VECDB_TYPE
  )


2024-09-26 19:15:11.934 | INFO     | utils:getEmbeddings:108 - Loaded local embeddings: textgain/allnli-GroNLP-bert-base-dutch-cased
2024-09-26 19:15:11.934 | INFO     | llm_class.llm_class:__init__:38 - Use Local LLM
2024-09-26 19:15:11.935 | INFO     | llm_class.llm_class:__init__:39 - Retrieving gemma2
2024-09-26 19:15:11.935 | INFO     | llm_class.llm_class:__init__:41 - Using local api url http://127.0.0.1:11434
2024-09-26 19:15:11.935 | INFO     | llm_class.llm_class:__init__:52 - Retrieved gemma2


In [36]:
ingester.ingest(mode=IngestionMode.question_answer_per_page, forceRebuild=True, addedMetaDataURLCSV="docs/metadata.csv")

2024-09-26 19:15:13.125 | INFO     | utils:getEmbeddings:108 - Loaded local embeddings: textgain/allnli-GroNLP-bert-base-dutch-cased
2024-09-26 19:15:13.126 | INFO     | ingest.ingester:ingest:137 - Skipping ingestion of file .DS_Store because it has extension tore
2024-09-26 19:15:13.126 | INFO     | ingest.ingester:ingest:172 - Vector store to be created for folder ./docs/kamerVragen
2024-09-26 19:15:13.185 | INFO     | ingest.ingester:ingest:185 - Files are added, so vector store for ./docs/kamerVragen needs to be updated
2024-09-26 19:15:13.194 | INFO     | ingest.file_parser:parse_pdf:50 - Extracting metadata
2024-09-26 19:15:13.195 | INFO     | ingest.file_parser:parse_pdf:53 - None
2024-09-26 19:15:13.195 | INFO     | ingest.file_parser:parse_pdf:55 - Extracting pages
2024-09-26 19:15:13.203 | INFO     | ingest.ingester:ingest:205 - No data found for id 0a6318e8-2a44-4331-a8e6-5e89d1c9b034.
2024-09-26 19:15:13.203 | INFO     | ingest.ingest_utils:clean_text:49 - Cleaning text of

In [37]:
querier.make_chain(folderSelected, my_vectordb_folder_path_selected)

2024-09-26 19:15:17.837 | INFO     | query.querier:make_chain:101 - Loaded chromadb from folder ./vector_stores/kamerVragen_chromadb_1024_256_local_embeddings_textgain/allnli-GroNLP-bert-base-dutch-cased
2024-09-26 19:15:17.837 | INFO     | query.querier:make_chain:126 - Executed Querier.make_chain


In [38]:
# querier.ask_question("""
#                      Kwamen er 82 treinen stil te staan tijdens hun rit?
#                      """, mode=EnumMode.metadata, system_prompt_override=SYSTEM_PROMPT)

## Place folder contents in json file

In [39]:


# from chunker import clean_pages
# from ingest.file_parser import FileParser
# import json 

# processed = []
# file_parser = FileParser()

# # Process files
# for file in os.listdir(my_folder_path_selected):
#   if not file.endswith(".pdf"):
#     continue
#   print("start processing file: ", file)
#   file_path = os.path.join(my_folder_path_selected, file)
#   raw_pages, _ = file_parser.parse_file(file_path)
#   cleaned_pages = clean_pages(raw_pages)
#   processed.extend(cleaned_pages)

# # Write processed pages to json file
# print("writing processed pages to json file")
# output_file = os.path.join(my_folder_path_selected, "processed.json")

# with open(output_file, "w") as f:
#   json.dump(processed, f)

# print("done")
  



## Evaluations

### Check if one single file can be retrived

In [40]:
querier.test_retrival(my_folder_path_selected, ingester)

2024-09-26 19:15:17.847 | INFO     | ingest.file_parser:parse_pdf:50 - Extracting metadata
2024-09-26 19:15:17.848 | INFO     | ingest.file_parser:parse_pdf:53 - None
2024-09-26 19:15:17.848 | INFO     | ingest.file_parser:parse_pdf:55 - Extracting pages
2024-09-26 19:15:17.873 | INFO     | query.querier:get_documents_with_scores:130 - Topscore most similar docs: 0.9805711507797241
2024-09-26 19:15:17.886 | INFO     | query.querier:get_documents_with_scores:130 - Topscore most similar docs: 0.9805711507797241
2024-09-26 19:15:17.887 | INFO     | query.querier:test_retrival_singular:210 - Highest score: 0.9805711507797241
2024-09-26 19:15:17.887 | INFO     | query.querier:test_retrival_singular:212 - Source document found
2024-09-26 19:15:17.900 | INFO     | query.querier:get_documents_with_scores:130 - Topscore most similar docs: 0.999999463558197
2024-09-26 19:15:17.913 | INFO     | query.querier:get_documents_with_scores:130 - Topscore most similar docs: 0.999999463558197
2024-09-26 